Yine titanicdeki modelimiz:  
Modelinizin performansı çapraz doğrulama yaparak ve farklı metrikler kullanarak değerlendirin.  
Hiperparemetre ayarlaması yaparak en uygun parametrelere sahip modeli belirleyin.  

In [39]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

In [40]:
df=pd.read_csv('train.csv')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [42]:
df['Age']=df['Age'].interpolate()

In [43]:
X=df[['Pclass','Sex','Age','SibSp','Parch']]
y=df['Survived']

In [44]:
X = pd.get_dummies(data=X, drop_first=True)

In [45]:
X_eğitim, X_test, y_eğitim, y_test =  train_test_split(X, y, test_size=0.20, random_state=111)

In [46]:
log_reg = LogisticRegression()

In [47]:
log_reg.fit(X_eğitim, y_eğitim)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

Aynı model için performans metriklerini 3. kısımdaki ödevde yaptık zaten. Farklı bir şey yok sanıyorum.  
Bir tek cross validation yapmalı ek olarak.

In [48]:
#k-fold cross validation kısmı baslıyor.

In [49]:
kf = KFold(n_splits=5, shuffle=True, random_state=1111)

In [50]:
parcalar = kf.split(X)
for num, (eğitim_index, test_index) in enumerate(parcalar):
    print("{}.Eğitim Kümesi Boyutu : {}".format(num+1,len(eğitim_index)))
    print("{}.Test Kümesi Boyutu   : {}".format(num+1,len(test_index)))
    print('-'*27)

1.Eğitim Kümesi Boyutu : 712
1.Test Kümesi Boyutu   : 179
---------------------------
2.Eğitim Kümesi Boyutu : 713
2.Test Kümesi Boyutu   : 178
---------------------------
3.Eğitim Kümesi Boyutu : 713
3.Test Kümesi Boyutu   : 178
---------------------------
4.Eğitim Kümesi Boyutu : 713
4.Test Kümesi Boyutu   : 178
---------------------------
5.Eğitim Kümesi Boyutu : 713
5.Test Kümesi Boyutu   : 178
---------------------------


In [51]:
lrm = LogisticRegression()
cv = cross_validate(estimator=lrm,
                     X=X,
                     y=y,
                     cv=10,return_train_score=True
                    )
print('Test Skorları            : ', cv['test_score'], sep = '\n')
print("-"*50)
print('Eğitim Skorları          : ', cv['train_score'], sep = '\n')

Test Skorları            : 
[0.78888889 0.77777778 0.75280899 0.82022472 0.78651685 0.78651685
 0.80898876 0.74157303 0.82022472 0.79545455]
--------------------------------------------------
Eğitim Skorları          : 
[0.79151061 0.78901373 0.79551122 0.78802993 0.78927681 0.79551122
 0.79426434 0.79925187 0.79177057 0.79701121]


In [52]:
print('Test Kümesi   Ortalaması : ', cv['test_score'].mean())
print('Eğitim Kümesi Ortalaması : ', cv['train_score'].mean())

Test Kümesi   Ortalaması :  0.7878975144705482
Eğitim Kümesi Ortalaması :  0.7931151512639151


In [53]:
cv = cross_validate(estimator=lrm, 
                     X=X,
                     y=y,
                     cv=10,
                     scoring = ['accuracy', 'precision', 'r2'],return_train_score=True
                    )

In [54]:
print('Test Kümesi Doğruluk Ortalaması     : {:.2f}'.format(cv['test_accuracy'].mean()))
print('Test Kümesi R-kare  Ortalaması      : {:.2f}'.format(cv['test_r2'].mean()))
print('Test Kümesi Hassasiyet Ortalaması   : {:.2f}'.format(cv['test_precision'].mean()))
print('Eğitim Kümesi Doğruluk Ortalaması   : {:.2f}'.format(cv['train_accuracy'].mean()))
print('Eğitim Kümesi R-kare  Ortalaması    : {:.2f}'.format(cv['train_r2'].mean()))
print('Eğitim Kümesi Hassasiyet Ortalaması : {:.2f}'.format(cv['train_precision'].mean()))

Test Kümesi Doğruluk Ortalaması     : 0.79
Test Kümesi R-kare  Ortalaması      : 0.10
Test Kümesi Hassasiyet Ortalaması   : 0.76
Eğitim Kümesi Doğruluk Ortalaması   : 0.79
Eğitim Kümesi R-kare  Ortalaması    : 0.13
Eğitim Kümesi Hassasiyet Ortalaması : 0.76


In [55]:
#hiperparametre ayarlaması kısmı

In [56]:
#Gridsearch
parametreler = {"C": [10 ** x for x in range (-5, 5, 1)],
                "penalty": ['l1', 'l2']
                }

In [57]:
grid_cv = GridSearchCV(estimator=LogisticRegression(),
                       param_grid = parametreler,
                       cv = 10
                      )
grid_cv.fit(X_eğitim, y_eğitim)

print("En iyi parametreler : ", grid_cv.best_params_)
print("En iyi skor         : ", grid_cv.best_score_)

En iyi parametreler :  {'C': 0.1, 'penalty': 'l2'}
En iyi skor         :  0.7949438202247191


In [58]:
sonuclar = grid_cv.cv_results_
df = pd.DataFrame(sonuclar)
df = df[['param_penalty','param_C', 'mean_test_score']]
df = df.sort_values(by='mean_test_score', ascending = False)

df.head()

,param_penalty,param_C,mean_test_score
9,l2,0.1,0.794944
19,l2,10000,0.790730
12,l1,10,0.790730
18,l1,10000,0.790730
17,l2,1000,0.790730


In [59]:
#randomizedsearch

In [60]:
parametreler = {"C": [10 ** x for x in range (-5, 5, 1)],
                "penalty": ['l1', 'l2']
                }

In [61]:
rs_cv = RandomizedSearchCV(estimator=LogisticRegression(),
                           param_distributions = parametreler,
                           cv = 10,
                           n_iter = 10,
                           random_state = 111,
                           scoring = 'precision'
                      )
rs_cv.fit(X_eğitim, y_eğitim)

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='warn', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='warn', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1,
                                              1, 10, 100, 1000, 10000],
                                        'penalty': ['l1', 'l2']},
                   pre_dispatch='2*n_jobs', random_sta

In [62]:
print("En iyi parametreler        : ", rs_cv.best_params_)
print("Tüm hassasiyet değerleri   : ", rs_cv.cv_results_['mean_test_score'])
print("En iyi hassasiyet değeri   : ", rs_cv.best_score_)

En iyi parametreler        :  {'penalty': 'l1', 'C': 100}
Tüm hassasiyet değerleri   :  [0.         0.         0.74368227 0.74299428 0.74826214 0.74826214
 0.         0.         0.74826214 0.        ]
En iyi hassasiyet değeri   :  0.748262142383516


In [63]:
sonuçlar_rs = rs_cv.cv_results_
df_rs = pd.DataFrame(sonuçlar_rs)
df_rs = df_rs[['param_penalty','param_C', 'mean_test_score']]
df_rs = df_rs.sort_values(by='mean_test_score', ascending = False)
df_rs

,param_penalty,param_C,mean_test_score
4,l1,100,0.748262
5,l2,1000,0.748262
8,l1,1000,0.748262
2,l2,10,0.743682
3,l2,1,0.742994
0,l2,0.0001,0.000000
1,l2,0.001,0.000000
6,l1,0.01,0.000000
7,l1,0.001,0.000000
9,l1,1e-05,0.000000
